In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!cd "/content/drive/MyDrive/Projects/Clubs/Analytics/Coord Projects/Model Zoo/PSPNet" && wget http://data.csail.mit.edu/places/ADEchallenge/ADEChallengeData2016.zip

In [ ]:
#!cd "/content/drive/MyDrive/Projects/Clubs/Analytics/Coord Projects/Model Zoo/PSPNet/datasets" && unzip ../ADEChallengeData2016.zip

In [ ]:
PATH = "/content/drive/MyDrive/Projects/Clubs/Analytics/Coord Projects/Model Zoo/PSPNet/datasets/ADEChallengeData2016"

---

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
import torch
from torchvision import transforms
import os, glob
import cv2
import numpy as np

class CreateDataset(torch.utils.data.Dataset):
    def __init__(self, PATH, mode='training', n_classes=150):
        self.mode = mode
        self.n_classes = n_classes
        self.entry = np.array([os.path.splitext(os.path.basename(entry))[0] for entry in glob.glob(os.path.join(PATH, "images", self.mode, "*.jpg"))])
        np.random.shuffle(self.entry)
        if self.mode == 'training':
            max_size = 16384
        elif self.mode == 'validation':
            max_size = 128
        self.entry = self.entry[:max_size]
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    def image_transform(self, image):
        image = np.float32(np.array(image))/255.
        image = image.transpose((2, 0, 1))
        image = self.normalize(torch.from_numpy(image.copy()))
        return image
    def label_transform(self, label):
        label = self.encode_label(label)
        label = label.transpose((2, 0, 1))
        return torch.from_numpy(label)

    def encode_label(self, label):
        encoded = np.zeros((label.shape[0], label.shape[1], self.n_classes))
        for row in range(label.shape[0]):
            for col in range(label.shape[1]):
                idx = label[row][col]-1
                encoded[row][col][idx] = 1
        return encoded

    def __getitem__(self, index):
        image = cv2.imread(os.path.join(PATH, "images", self.mode, str(self.entry[index]+ ".jpg")), 1)
        label = cv2.imread(os.path.join(PATH, "annotations", self.mode, str(self.entry[index]+ ".png")), 0)
        image = cv2.resize(image, (224,224), interpolation=cv2.INTER_AREA)
        label = cv2.resize(label, (224,224), interpolation=cv2.INTER_NEAREST)
        image = self.image_transform(image)
        label = self.label_transform(label)
        return {'image': image, 'label': label}

    def __len__(self):
        return len(self.entry)

In [ ]:
PATH = "/content/drive/MyDrive/Projects/Clubs/Analytics/Coord Projects/Model Zoo/PSPNet/datasets/ADEChallengeData2016"

train_data = CreateDataset(PATH, mode='training')
val_data = CreateDataset(PATH, mode='validation')

In [ ]:
print(len(train_data))
print(len(val_data))

16384
128


---

In [ ]:
!cp "/content/drive/MyDrive/Projects/Clubs/Analytics/Coord Projects/Model Zoo/PSPNet/scripts/resnet.py" /content/resnet.py

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

from resnet import resnet101, resnet50

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=1, padding=0, stride=1, dilation=1, bias=False):
        super(ConvBlock, self).__init__()
        padding = (kernel_size + (kernel_size - 1) * (dilation - 1)) // 2
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, bias=bias),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        out = self.conv(x)
        return out

def upsample(input, size=None, scale_factor=None, align_corners=False):
    out = F.interpolate(input, size=size, scale_factor=scale_factor, mode='bilinear', align_corners=align_corners)
    return out

In [ ]:
class PyramidPooling(nn.Module):
    def __init__(self, in_channels):
        super(PyramidPooling, self).__init__()
        self.pooling_size = [1, 2, 3, 6]
        self.channels = in_channels // 4

        self.pool1 = nn.Sequential(
            nn.AdaptiveAvgPool2d(self.pooling_size[0]),
            ConvBlock(in_channels, self.channels, kernel_size=1),
        )

        self.pool2 = nn.Sequential(
            nn.AdaptiveAvgPool2d(self.pooling_size[1]),
            ConvBlock(in_channels, self.channels, kernel_size=1),
        )

        self.pool3 = nn.Sequential(
            nn.AdaptiveAvgPool2d(self.pooling_size[2]),
            ConvBlock(in_channels, self.channels, kernel_size=1),
        )

        self.pool4 = nn.Sequential(
            nn.AdaptiveAvgPool2d(self.pooling_size[3]),
            ConvBlock(in_channels, self.channels, kernel_size=1),
        )

    def forward(self, x):
        out1 = self.pool1(x)
        out1 = upsample(out1, size=x.size()[-2:])

        out2 = self.pool2(x)
        out2 = upsample(out2, size=x.size()[-2:])

        out3 = self.pool3(x)
        out3 = upsample(out3, size=x.size()[-2:])

        out4 = self.pool4(x)
        out4 = upsample(out4, size=x.size()[-2:])

        out = torch.cat([x, out1, out2, out3, out4], dim=1)

        return out

In [ ]:
class PSPNet(nn.Module):
    def __init__(self, n_classes=150):
        super(PSPNet, self).__init__()
        self.out_channels = 2048

        self.backbone = resnet50(pretrained=True)
        self.stem = nn.Sequential(
            *list(self.backbone.children())[:4],
        )
        self.block1 = self.backbone.layer1
        self.block2 = self.backbone.layer2
        self.block3 = self.backbone.layer3
        self.block4 = self.backbone.layer4
        self.low_level_features_conv = ConvBlock(512, 64, kernel_size=3)

        self.depth = self.out_channels // 4
        self.pyramid_pooling = PyramidPooling(self.out_channels)

        self.decoder = nn.Sequential(
            ConvBlock(self.out_channels * 2, self.depth, kernel_size=3),
            nn.Dropout(0.1),
            nn.Conv2d(self.depth, n_classes, kernel_size=1),
        )

        self.aux = nn.Sequential(
            ConvBlock(self.out_channels // 2, self.depth // 2, kernel_size=3),
            nn.Dropout(0.1),
            nn.Conv2d(self.depth // 2, n_classes, kernel_size=1),
        )

    def forward(self, image, label=None):
        out = self.stem(image)
        out1 = self.block1(out)
        out2 = self.block2(out1)
        out3 = self.block3(out2)
        aux_out = self.aux(out3)
        aux_out = upsample(aux_out, size=image.size()[-2:], align_corners=True)
        out4 = self.block4(out3)

        out = self.pyramid_pooling(out4)
        out = self.decoder(out)
        out = upsample(out, size=image.size()[-2:])

        out = upsample(out, size=image.size()[-2:], align_corners=True)
        #out = F.softmax(out, dim=1)

        return out

---

In [ ]:
from torch.utils.data import DataLoader
import torch.optim as optim
import gc
import time

import matplotlib.pyplot as plt

In [ ]:
batch_size = 16
start_epochs = 0
epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CHECKPOINT = "/content/drive/MyDrive/Projects/Clubs/Analytics/Coord Projects/Model Zoo/PSPNet/checkpoints"

In [ ]:
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [ ]:
model = PSPNet()
model.to(device)

resnet50-19c8e357.pth: 103MB [00:00, 113MB/s]                            


PSPNet(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [ ]:
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
if os.path.exists(os.path.join(CHECKPOINT, "model.pth")):
    checkpoints = torch.load(os.path.join(CHECKPOINT, "model.pth"))

    model.load_state_dict(checkpoints['model_state_dict'])
    optimizer.load_state_dict(checkpoints['optimizer_state_dict'])
    start_epochs = checkpoints['epoch']

In [ ]:
train_loss = []
train_acc = []
val_loss = []
val_acc = []

In [ ]:
def plot_loss(epoch, train_loss, val_loss):
    epochs = np.arange(len(train_loss))
    plt.plot(epochs, train_loss, label="Train Loss", color="green")
    plt.plot(epochs, val_loss, label="Validation Loss", color="red")

    plt.title("Training Loss - "+str(epoch))
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(loc="best")

    plt.savefig(os.path.join(CHECKPOINT, "Loss.png"))
    plt.close()

In [ ]:
for epoch in range(start_epochs+1, epochs+start_epochs+1):
    print("Starting Epoch[{0}/{1}]".format(epoch, epochs+start_epochs))
    
    time_epoch_start = time.time()
    gc.collect()
    torch.cuda.empty_cache()

    model.train()
    epoch_train_loss = []
    epoch_val_loss = []
    for idx, batch in enumerate(trainloader, 1):
        time_batch_start = time.time()
        image = batch['image'].to(device)
        label = batch['label'].to(device)
        preds = model(image)
        loss = criterion(preds, label)
        
        loss.backward()
        optimizer.step()
        epoch_train_loss.append(loss.item())
        time_batch_end = time.time()
        print("Epoch[{0}]: Batch[{1}]   Train Loss: {2}     Time: {3}".format(epoch, idx, loss.item(), time_batch_end-time_batch_start))
        gc.collect()
        torch.cuda.empty_cache()

    train_loss.append(epoch_train_loss[-1])

    gc.collect()
    torch.cuda.empty_cache()

    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(valloader, 1):
            time_batch_start = time.time()
            image = batch['image'].to(device)
            label = batch['label'].to(device)
            preds = model(image)
            loss = criterion(preds, label)

            epoch_val_loss.append(loss.item())
            time_batch_end = time.time()
            print("Epoch[{0}]: Batch[{1}]   Val Loss: {2}     Time: {3}".format(epoch, idx, loss.item(), time_batch_end-time_batch_start))

        val_loss.append(epoch_val_loss[-1])

    gc.collect()
    torch.cuda.empty_cache()

    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss[-1],
            }, os.path.join(CHECKPOINT, "model.pth"))
    
    plot_loss(epoch, train_loss, val_loss)

    time_epoch_end = time.time()
    print("Finished Epoch[{0}/{1}] in Time: {2}".format(epoch, epochs+start_epochs, time_epoch_end-time_epoch_start))

Starting Epoch[5/6]


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch[5]: Batch[1]   Train Loss: 0.029129930609841665     Time: 1.4080426692962646
Epoch[5]: Batch[2]   Train Loss: 0.028919635106608964     Time: 1.3773972988128662
Epoch[5]: Batch[3]   Train Loss: 0.030620687727820756     Time: 1.3787586688995361
Epoch[5]: Batch[4]   Train Loss: 0.028615320370407125     Time: 1.3880276679992676
Epoch[5]: Batch[5]   Train Loss: 0.02991612826465397     Time: 1.3839471340179443
Epoch[5]: Batch[6]   Train Loss: 0.025937067124050518     Time: 1.3778319358825684
Epoch[5]: Batch[7]   Train Loss: 0.02735940616616979     Time: 1.3742187023162842
Epoch[5]: Batch[8]   Train Loss: 0.028439698274732687     Time: 1.3883349895477295
Epoch[5]: Batch[9]   Train Loss: 0.026399859088310666     Time: 1.3877308368682861
Epoch[5]: Batch[10]   Train Loss: 0.02785656333546975     Time: 1.36895751953125
Epoch[5]: Batch[11]   Train Loss: 0.028080850275465915     Time: 1.3809592723846436
Epoch[5]: Batch[12]   Train Loss: 0.02446043168081937     Time: 1.3722093105316162
Epoch[5